<a href="https://colab.research.google.com/github/akamrume328/tennisvision/blob/feature%2F%231/notebooks/2_model_training_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Google ColabでのYOLOv8モデルのトレーニング

このノートブックは、Google Driveに保存されたカスタムデータセットを使用して、Google Colab上でYOLOv8モデルをトレーニングするためのものです。
以下の内容をカバーします：
1. Google Driveのマウント。
2. `ultralytics`ライブラリのインストール。
3. **データセットの準備とGoogle Driveへのアップロード**:
   - YOLOv8形式のデータセット（画像ファイル群、対応するラベルファイル群、そしてデータセット設定ファイル `data.yaml`）を準備します。
   - `data.yaml` には、クラス名、クラス数、そして訓練データと検証データへのパスを記述します。これらのパスは、画像とラベルが格納されているディレクトリを指すか、あるいは訓練用/検証用の画像ファイルパスをリストしたテキストファイル（例: `train.txt`, `val.txt`）を指します。
   - `train.txt` や `val.txt` を使用する場合、これらは1行に1つの画像ファイルへのパス（例: `images/train/frame_000001.png` や、`data.yaml` からの相対パスで `../images/frame_000001.png` など）を記述したテキストファイルです。これらのテキストファイルと、対応する画像およびラベルファイルは、`data.yaml` から相対的にアクセスできる位置に配置する必要があります。
   - 準備したデータセット全体（画像、ラベル、`data.yaml`、`train.txt`、`val.txt` 等）をGoogle Driveの任意の場所にアップロードします。
4. データセットとプロジェクト出力のパス設定（Google Drive上の `data.yaml` へのパスを指定）。
5. YOLOv8モデルのトレーニング。
6. トレーニング済みモデルと結果の保存場所に関する情報。

In [1]:
# Google Driveをマウント
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## PyTorch XLAのインストール (TPU利用時)

Google ColabでTPUを使用してPyTorchベースのモデル（YOLOv8など）をトレーニングする場合、`torch_xla` ライブラリが必要です。
このライブラリは、PyTorchがTPUハードウェアと通信するためのコンポーネントを提供します。

以下のセルを実行して、`torch_xla` をインストールしてください。
**注意:** TPUランタイムを使用している場合にのみ、このインストールが意味を持ちます。GPUまたはCPUランタイムでは不要です。

In [ ]:
# PyTorch XLAライブラリをインストール (TPU利用時に必要)
# ColabのTPUランタイムでは、対応するPyTorchのバージョンと共にインストールされることが多いですが、
# 明示的にインストールすることで互換性を確保します。
# !pip install torch_xla cloud-tpu-client https://storage.googleapis.com/tpu-pytorch/wheels/colab/torch_xla-2.0-cp310-cp310-linux_x86_64.whl --force-reinstall
# ↑ ColabのPythonバージョンやPyTorchのバージョンによって適切なwheelが異なる場合があります。
# ultralyticsが依存するPyTorchバージョンと互換性のあるtorch_xlaをインストールするのが望ましいです。
# まずはシンプルにpip installしてみます。
# ultralyticsのインストール時にPyTorchもインストールされるため、その後にtorch_xlaをインストールするか、
# ultralyticsがTPU環境を検知して適切に処理することを期待します。
# ここでは、ultralyticsの前にインストールを試みます。

import os
import torch # PyTorchが既に存在するか確認するため

# TPUが利用可能かどうかの確認（オプションですが、TPUランタイムでのみ実行する目安）
try:
    if 'COLAB_TPU_ADDR' in os.environ:
        print("TPU環境を検出しました。torch_xlaをインストールします。")
        # 最新の推奨コマンドは変更されることがあるため、公式ドキュメントも参照してください。
        # https://github.com/pytorch/xla
        # !pip install torch~=2.1.0 torch_xla~=2.1.0 torchvision~=0.16.0 torchaudio~=2.1.0 torchtext --index-url https://download.pytorch.org/whl/cu121 -q
        # ↑はGPU用。TPU用は以下のような形式になることが多い。
        # !pip install cloud-tpu-client
        # !pip install torch_xla[tpu]~=2.1.0 -f https://storage.googleapis.com/libtpu-releases/index.html
        
        # ultralyticsがPyTorch 2.1系を推奨している場合があるため、それに合わせる
        # ultralyticsのドキュメントでは、TPUセットアップについて特別なpip install指示は最近見られない。
        # device='tpu'で内部的に処理されることを期待。
        # ただし、明示的なインストールが必要なケースも過去にはあった。
        # ひとまず、ultralyticsのインストールに任せ、もしTPUエラーが出たらここに戻ってtorch_xlaのインストールを試す。
        print("torch_xlaの明示的なインストールはコメントアウトしています。")
        print("ultralyticsが内部でTPU設定を処理することを期待します。")
        print("もしTPUでのトレーニングに失敗する場合、このセルで適切なtorch_xlaのインストールコマンドを有効化してください。")
        print("例: !pip install torch_xla")
        # !pip install torch_xla -q
    else:
        print("TPU環境は検出されませんでした。torch_xlaのインストールはスキップします。")
        print("(GPUまたはCPUランタイムの可能性があります)")
except ImportError:
    print("PyTorchがまだインストールされていないようです。ultralyticsのインストール時に導入されるはずです。")
except Exception as e:
    print(f"TPU環境の確認またはtorch_xlaインストール中にエラー: {e}")

# ultralyticsのドキュメントでは、TPU使用時に特別なライブラリインストール手順は明記されていません（2023年後半以降）。
# `device='tpu'` を指定することで、ultralyticsが内部的にPyTorch XLAの利用を試みる可能性があります。
# このセルは、もし `ultralytics` の標準インストールだけではTPUが動作しない場合のトラブルシューティング用として残します。

In [ ]:
!pip install torch_xla -q

In [ ]:
# ultralyticsライブラリをインストール
!pip install ultralytics

## Gitのセットアップ (オプション)

Colabから直接Gitリポジトリにコミットやプッシュを行いたい場合、以下のセルでGitのユーザー名とメールアドレスを設定します。
これは、コミット履歴に記録される作成者情報となります。

**注意:** ここで設定するメールアドレスやユーザー名は公開される可能性があるため、取り扱いに注意してください。

In [ ]:
# Gitのユーザー名とメールアドレスを設定
# 以下の "Your Name" と "youremail@example.com" をご自身のものに置き換えてください。
!git config --global user.name "akamrume328" # ★★★ あなたの名前に置き換えてください ★★★
!git config --global user.email "akamarume@icloud.com" # ★★★ あなたのメールアドレスに置き換えてください ★★★

print("Gitのユーザー名とメールアドレスが設定されました。")
!git config --global --list # 設定内容の確認

Gitのユーザー名とメールアドレスが設定されました。
user.name=akamrume328
user.email=akamarume@icloud.com


## パスの設定

データセットの`data.yaml`ファイルと、トレーニング結果（例: モデルの重み、ログ）を保存するディレクトリのパスを定義します。

## Google Drive上のデータセットを展開 (オプション)

もしデータセットがGoogle Drive上でZIPファイルとして保存されている場合、以下のセルを実行してColabのローカルストレージに展開できます。
これにより、トレーニング中のファイルアクセスが高速になることがあります。
ZIPファイルには、画像ファイル、ラベルファイル、`data.yaml`、そして必要に応じて `train.txt` や `val.txt` など、データセット全体が含まれていることを想定しています。

**注意:**
- ZIPファイルへのパス (`zip_file_path_on_drive`) と展開先のディレクトリ (`colab_dataset_dir`) をご自身の環境に合わせて設定してください。
- データセットが大きい場合、展開に時間がかかることがあります。
- Colabのセッションが終了すると、展開されたデータは削除されます。再度ノートブックを実行する際には、このセルも再実行する必要があります。
- このセルを実行した場合、次の「パスの設定」セルで `dataset_yaml_path` が、Colab上に展開されたデータセット内の `data.yaml` を指すように必ず更新してください。

In [ ]:
import zipfile
import os

# --- ★★★ ユーザー設定項目 ★★★ ---
# Google Drive上のZIPファイルのパス
# 例: '/content/drive/MyDrive/datasets/my_dataset.zip'
# このZIPファイルには、画像、ラベル、data.yaml、train.txt、val.txtなどが含まれていることを想定しています。
zip_file_path_on_drive = '/content/drive/MyDrive/datasets/dataset.zip'  # ★★★ あなたのZIPファイルのパスに置き換えてください ★★★

# Colab上にデータセットを展開するディレクトリ
# 通常、'/content/' 以下に作成します。
colab_dataset_dir = '/content/datasets' # ★★★ 必要であれば変更してください ★★★

# --- ★★★ 設定項目終了 ★★★ ---

# 展開先ディレクトリを作成
if not os.path.exists(colab_dataset_dir):
    os.makedirs(colab_dataset_dir)
    print(f"展開先ディレクトリを作成しました: {colab_dataset_dir}")
else:
    print(f"展開先ディレクトリは既に存在します: {colab_dataset_dir}")

if os.path.exists(zip_file_path_on_drive):
    print(f"ZIPファイルが見つかりました: {zip_file_path_on_drive}")
    print(f"ファイルを {colab_dataset_dir} に展開しています...")
    try:
        with zipfile.ZipFile(zip_file_path_on_drive, 'r') as zip_ref:
            zip_ref.extractall(colab_dataset_dir)
        print(f"データセットの展開が完了しました。展開先: {colab_dataset_dir}")

        print("\n--- 展開後の確認と次のステップ --- ")
        print(f"展開先ディレクトリ ({colab_dataset_dir}) の内容:")
        extracted_items = os.listdir(colab_dataset_dir)
        if extracted_items:
            for item in extracted_items:
                print(f"- {item}")
            # 展開されたディレクトリ内に data.yaml があるか確認する (より親切なガイド)
            # 例えば、ZIPが 'my_dataset_root/data.yaml' のように展開される場合がある
            possible_yaml_paths = []
            for root, dirs, files in os.walk(colab_dataset_dir):
                if "data.yaml" in files:
                    possible_yaml_paths.append(os.path.join(root, "data.yaml"))
            
            if possible_yaml_paths:
                print("\n展開された可能性のある data.yaml のパス:")
                for p_yaml in possible_yaml_paths:
                    print(f"- {p_yaml}")
                print(f"これらのいずれかを次の「パスの設定」セルの `dataset_yaml_path` に設定してください。")
            else:
                print(f"\n警告: 展開されたディレクトリ内に 'data.yaml' が見つかりませんでした。")
                print(f"ZIPファイルの内容と展開後の構造を確認し、`data.yaml`への正しいパスを次のセルで設定してください。")

        else:
            print("(ディレクトリは空か、アクセスできませんでした)")

        print(f"\n重要: 次の「パスの設定」セルで、`dataset_yaml_path` が、")
        print(f"このColab上の展開先ディレクトリ ({colab_dataset_dir}) 内の `data.yaml` を指すように更新してください。")
        print(f'例えば、もしZIP展開後に "{os.path.join(colab_dataset_dir, "your_dataset_main_folder", "data.yaml")}" のような構造になる場合、')
        print(f'dataset_yaml_path = "{os.path.join(colab_dataset_dir, "your_dataset_main_folder", "data.yaml")}" のように設定します。')
        print("ZIPファイル内のフォルダ構造を確認し、適切にパスを修正してください。")

    except zipfile.BadZipFile:
        print(f"エラー: {zip_file_path_on_drive} は有効なZIPファイルではありません。")
    except Exception as e:
        print(f"展開中にエラーが発生しました: {e}")
else:
    print(f"エラー: ZIPファイルが見つかりません: {zip_file_path_on_drive}")
    print("Google Drive上のパスを確認してください。")
    print("もしZIP展開機能を使用しない場合は、このセルをスキップして問題ありません。")

In [ ]:
import os

# --- 重要: これらのパスをGoogle Driveの構造、またはZIP展開後のColab上の構造に合わせて設定してください ---

# データセットのdata.yamlファイルへのパス
# Google Driveから直接参照する場合の例: '/content/drive/MyDrive/datasets/my_yolo_dataset/data.yaml'
# 上のセルでZIPを展開した場合の例: '/content/datasets/my_yolo_dataset_in_zip/data.yaml'
# ★★★ この `data.yaml` は、訓練画像/ラベルの場所、クラス名、そして
# ★★★ `train: path/to/train.txt` や `val: path/to/val.txt` のように、
# ★★★ 訓練用/検証用テキストファイルへのパス（通常はdata.yamlからの相対パス）を定義している必要があります。
# ★★★ `train.txt` や `val.txt` は、画像ファイルへのパスのリスト（例: `images/train/frame_000001.png` や `../images/frame_000001.png`）を含みます。
dataset_yaml_paths = [
    '/content/datasets/dataset1/data.yaml',
    # '/content/datasets/dataset2/data.yaml',
    # ...必要に応じて追加...
]

experiment_names = [
    'tennis_detection_run1',
    # 'tennis_detection_run2',
    # ...必要に応じて追加...
]

checkpoint_to_resume_paths = [
    None,
    # '/content/drive/MyDrive/models/YOLOv8_Training_Outputs/tennis_detection_run2/weights/last.pt',
    # ...必要に応じて追加...
]


# リストの長さ検証 (例)
if not (len(dataset_yaml_paths) == len(experiment_names) == len(checkpoint_to_resume_paths)):
    print("エラー: リストの長さが一致しません。")
    print(f"dataset_yaml_paths: {len(dataset_yaml_paths)}")
    print(f"experiment_names: {len(experiment_names)}")
    print(f"checkpoint_to_resume_paths: {len(checkpoint_to_resume_paths)}")
else:
    print("リストの長さは一致しています。")

In [ ]:
from ultralytics import YOLO
import os # osモジュールをインポート (既に上のセルでインポートされている可能性あり)
from google.colab import runtime # ランタイム切断のためにインポート

# 新たに1つの統合データセットを指定
unified_dataset_yaml_path = '/content/datasets/unified_data.yaml'  # ★統合用のdata.yamlを指定

model_name = 'yolov8s.pt'
num_epochs = 100
batch_size = 16
img_size = 1920

# 以前のループを削除し、単一のトレーニング呼び出しに
model = YOLO(model_name)
print("単一のモデルに対して統合データセットで学習を開始します。")
model.train(
    data=unified_dataset_yaml_path,
    epochs=num_epochs,
    imgsz=img_size,
    batch=batch_size,
    project=project_output_dir,  # 以前の変数を流用
    name="unified_experiment",   # 統合データセットの実験名
    exist_ok=True,
    save_period=save_every_n_epochs if save_every_n_epochs > 0 else -1,
    device=training_device,
)
print("統合データセットのトレーニングが完了しました。")

In [ ]:
import gc
import os

# 1. Pythonのガベージコレクションを実行
gc.collect()
print("Pythonのガベージコレクションを実行しました。")

# 2. PyTorchのGPUキャッシュをクリア (ultralyticsは内部でPyTorchを使用)
try:
    import torch
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
        print("PyTorchのGPUキャッシュをクリアしました。")
    else:
        print("CUDAが利用できないため、PyTorchのGPUキャッシュクリアはスキップされました。")
except ImportError:
    print("PyTorchが見つからないため、GPUキャッシュのクリアはスキップされました。")
except Exception as e:
    print(f"PyTorch GPUキャッシュクリア中にエラーが発生しました: {e}")

# 3. pipのキャッシュをクリア
print("\npipのキャッシュをクリアしています...")
!pip cache purge
# 上のコマンドの出力がColabのセルに表示されます。
# 成功メッセージはpipのバージョンによって異なる場合があります。
print("pipのキャッシュクリアコマンドを実行しました。")


print("\n--- キャッシュクリア操作に関する補足 ---")
print("上記はいくつかの一般的なキャッシュクリア操作です。")
print("最も効果的にRAMとGPUメモリをリセットする方法は、Colabのメニューから")
print("「ランタイム」->「ランタイムを再起動」を選択することです。")
print("ただし、この操作を行うと、現在のセッションの変数や状態はすべて失われますのでご注意ください。")
print("トレーニングの最後にランタイムを切断する (`runtime.unassign()`) のも、リソース解放の確実な方法です。")


## トレーニング結果と保存されたモデル

トレーニング後、結果（メトリクス、混同行列、モデルの重みなど）は指定したディレクトリに保存されます：
`{project_output_dir}/{experiment_name}`

確認すべき主なファイル：
- **重み:** `weights`サブディレクトリ内 (例: `best.pt`, `last.pt`)
  - `best.pt`: 最良の検証メトリクス（通常はmAP50-95）を達成したモデルの重み。このモデルを推論に使用するのが一般的です。
  - `last.pt`: トレーニングの最終エポックのモデルの重み。
  - `epoch_XXX.pt` (例: `epoch_10.pt`, `epoch_20.pt`): `save_period` で指定されたエポックごとに保存されるチェックポイント。これらを使用して特定のエポックからトレーニングを再開したり、その時点でのモデル性能を評価したりできます。
- **結果CSV:** `results.csv`にはエポックごとのメトリクスの概要が含まれています。
- **プロット:** 混同行列、P-R曲線などのさまざまなプロット (例: `confusion_matrix.png`, `PR_curve.png`)

これらのファイルはGoogle Driveからダウンロードできます。

In [ ]:
# 複数の experiment_names に対応して結果を確認
for i, exp_name in enumerate(experiment_names):
    print(f"\n--- 結果確認 {i+1} 個目: {exp_name} ---")
    weights_dir = os.path.join(project_output_dir, exp_name, 'weights')
    if os.path.exists(weights_dir):
        print(f"重みディレクトリ: {weights_dir}")
        for f_name in os.listdir(weights_dir):
            print(f"- {f_name}")
    else:
        print(f"{weights_dir} が見つかりません")

    best_model_path = os.path.join(weights_dir, 'best.pt')
    print(f"最良モデル: {best_model_path}")
    if not os.path.exists(best_model_path):
        print("best.pt が見つかりませんでした")